In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf


from official.modeling import tf_utils
from official import nlp
from official.nlp import bert
from cubert_tokenizer import python_tokenizer, code_to_subtokenized_sentences

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks
from tensor2tensor.data_generators import text_encoder
import pandas as pd

import json
from sklearn import preprocessing
import itertools

MAX_ARG_LENGTH =20
DS_PATH = "../data/_all_data.csv"
EPOCHS = 3
shuffle_buffer_size = 1000
SEQ_LENGTH = 512

bert_tokenizer = bert.tokenization.FullTokenizer(
    vocab_file="../bert2/cuvocab.txt",
     do_lower_case=True)

with open("../bert2/cubert_config.json") as conf_file:
    config_dict = json.loads(conf_file.read())
    bert_config = bert.configs.BertConfig.from_dict(config_dict)

bert_encoder = bert.bert_models.get_transformer_encoder(
    bert_config, sequence_length=SEQ_LENGTH)

tf.keras.utils.plot_model(bert_encoder, show_shapes=True, dpi=48)
# tf.keras.utils.plot_model(bert_classifier, show_shapes=True, dpi=48)


checkpoint = tf.train.Checkpoint(model=bert_encoder)
checkpoint.restore('../bert2/bert1-1').assert_consumed()
if ".json" in DS_PATH:
    data = pd.read_json(DS_PATH)
else:
    data = pd.read_csv(DS_PATH)

tokenizer = python_tokenizer.PythonTokenizer()
subword_tokenizer = text_encoder.SubwordTextEncoder("../bert2/cuvocab.txt")

data_body1 = data['body.1']

## Preprocessign arg

data['arg_types'] = data['arg_types'].apply(eval)
df_labels = pd.DataFrame(data['arg_types'].values.tolist())

print(df_labels.head())

max_label_length=10
df_labels2=df_labels.iloc[:,0:max_label_length]
df_labels2[pd.isnull(df_labels2)]  = 'NaN'

enc = preprocessing.LabelEncoder()
all_types = df_labels.apply(pd.Series).stack().values
enc.fit(all_types)

df3 = df_labels2.apply(enc.transform)
data['labels'] = df3.values.tolist()




Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
         0     1     2     3     4     5     6     7     8     9   ...    51  \
0      dict  None  None  None  None  None  None  None  None  None  ...  None   
1      Task  None  None  None  None  None  None  None  None  None  ...  None   
2  datetime  None  None  None  None  None  None  None  None  None  ...  None   
3  datetime  None  None  None  None  None  None  None  None  None  ...  None   
4      Task  None  None  None  None  None  None  None  None  None  ...  None   

     52    53    54    55    56    57    58    59    60  
0  None  None  None  None  None  None  None  None  None  
1  None  None  None  None  None  None  None  None  None  
2  None  None  None  None  None  None  None  None  None  
3  None  None  None  None  None  None  None  None  None  
4  None  None  None  None  None  None  None  None  None  

[5 rows x 61 columns]


In [4]:
tf.sequence_mask([123], 512)

<tf.Tensor: shape=(1, 512), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  

In [2]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb_result = mlb.fit_transform(df_labels2.values)
df3 = pd.DataFrame(mlb_result)

In [25]:
data

,author,repo,file,has_type,body,name,docstring,func_descr,arg_names,arg_types,arg_descrs,return_type,return_expr,return_descr,body.1,args_occur,arg_names_len,arg_types_len,labels
0,Fongshway,twpm,./cloner_output/Fongshway__twpm/twpm/hook_runn...,https://github.com/Fongshway/twpm,True,to output,convert serialize task representation taskwarr...,convert serialize task representation taskwarr...,['task'],[dict],['serialize task'],str,['json dump task separator'],taskwarrior json,@staticmethod\n def to_output(task: dic...,['v task item'],1,1,"[14890, 7480, 7480, 7480, 7480, 7480, 7480, 74..."
1,Fongshway,twpm,./cloner_output/Fongshway__twpm/twpm/hooks/exa...,https://github.com/Fongshway/twpm,True,main,example hook entry point param task task insta...,example hook entry point,['task'],[Task],['task instance'],None,[],none,"def main(task: Task) -> None:\n """"""\n ex...",['original task description task description o...,1,1,"[11429, 7480, 7480, 7480, 7480, 7480, 7480, 74..."
2,Fongshway,twpm,./cloner_output/Fongshway__twpm/twpm/hooks/def...,https://github.com/Fongshway/twpm,True,be local midnight,helper function evaluate whether dateime midni...,helper function evaluate whether dateime midni...,['timestamp'],[datetime],[''],bool,['timestamp astimezone tz tzlocal time time'],boolean indicating datetime midnight local time,def is_local_midnight(timestamp: datetime) -> ...,[],1,1,"[14848, 7480, 7480, 7480, 7480, 7480, 7480, 74..."
3,Fongshway,twpm,./cloner_output/Fongshway__twpm/twpm/hooks/def...,https://github.com/Fongshway/twpm,True,set default time,helper function set default timestamp give dat...,helper function set default timestamp give dat...,['timestamp'],[datetime],[''],datetime,['timestamp replace hour default time hour min...,datetime hour minute second value set default,def set_default_time(timestamp: datetime) -> d...,[],1,1,"[14848, 7480, 7480, 7480, 7480, 7480, 7480, 74..."
4,Fongshway,twpm,./cloner_output/Fongshway__twpm/twpm/hooks/def...,https://github.com/Fongshway/twpm,True,main,default time hook entry point param task task ...,default time hook entry point,['task'],[Task],['task instance'],None,"['return', 'return']",none,def main(task: Task) -> None:\n # pylint: d...,['task due date task get task due task wait da...,1,1,"[11429, 7480, 7480, 7480, 7480, 7480, 7480, 74..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125716,WIPACrepo,lta,./cloner_output/WIPACrepo__lta/tests/test_loca...,https://github.com/WIPACrepo/lta,True,gen record,NaN,NaN,['i'],[int],[''],"Dict[str, Union[int, str, Dict[str, str], List...",['logical name f data exp ice cube filter pf f...,NaN,"def gen_record(i: int) -> Dict[str, Union[...",[],1,1,"[15209, 7480, 7480, 7480, 7480, 7480, 7480, 74..."
125717,WIPACrepo,lta,./cloner_output/WIPACrepo__lta/tests/test_pick...,https://github.com/WIPACrepo/lta,True,gen file,NaN,NaN,['i'],[int],[''],"Dict[str, str]",['logical name f data exp ice cube filter pf f...,NaN,"def gen_file(i: int) -> Dict[str, str]:\n ...",[],1,1,"[15209, 7480, 7480, 7480, 7480, 7480, 7480, 74..."
125718,WIPACrepo,lta,./cloner_output/WIPACrepo__lta/tests/test_pick...,https://github.com/WIPACrepo/lta,True,gen record,NaN,NaN,['i'],[int],[''],"Dict[str, Union[int, str, Dict[str, str], List...",['logical name f data exp ice cube filter pf f...,NaN,"def gen_record(i: int) -> Dict[str, Union[...",[],1,1,"[15209, 7480, 7480, 7480, 7480, 7480, 7480, 74..."
125719,WIPACrepo,lta,./cloner_output/WIPACrepo__lta/resources/rucio...,https://github.com/WIPACrepo/lta,True,print response,print interesting thing response object,print interesting thing response object,"['title', 'response']","[str, RucioResponse]","['', '']",None,[],NaN,"def print_response(title: str, response: Rucio...","['print title', 'dump response indent']",2,2,"[16099, 10062, 7480, 7480, 7480, 7480, 7480, 7..."


In [6]:
def transform_labels(df_labels_batch):
    df_labels = pd.DataFrame(df_labels_batch['arg_types'].values.tolist())
    df_labels[pd.isnull(df_labels_batch)] = 'NaN'
    df_labels = df_labels.apply(enc.transform)
    return mlb.transform(df_labels.values)

def transform(code_text):
    print(code_text)
    return [2]+sum(code_to_subtokenized_sentences.code_to_cubert_sentences(
        code=code_text,
        initial_tokenizer=tokenizer,
        subword_tokenizer=subword_tokenizer),[])


batch_size = 1
MAX_BODY_LENGTH = 64
def gen():
    for _, data_batch in data.groupby(np.arange(len(data))//batch_size):
        if len(data_batch) < batch_size:
            continue
        ids = []
        sentence1 = []
        for _,data_batch_i in data_batch.iterrows():
            id_list = []
            sentence2 = np.array(transform(data_batch_i['body.1'])[:SEQ_LENGTH-1]+[3])
            for label in eval(data_batch_i['arg_names']):
                print(sentence2, label, np.where(sentence2 == subword_tokenizer.encode(label)[0])[0][0])
                id_list.append( np.where(sentence2 == subword_tokenizer.encode(label)[0])[0][0])
            sentence1.append(sentence2)
        sentence1 = tf.ragged.constant(sentence1)
        sentence1 = sentence1.to_tensor(default_value=0, shape=[batch_size, SEQ_LENGTH])
        id_list =  tf.convert_to_tensor(id_list)
        # sentence1 = tf.reshape(sentence1, [batch_size,8*MAX_BODY_LENGTH])
        labels = tf.convert_to_tensor([np.array(data_batch_i['labels']) for _,data_batch_i in data_batch.iterrows()])
        type_s1 = tf.zeros_like(sentence1)
        yield ({'input_word_ids': sentence1,
            'input_mask': tf.ones_like(sentence1),
            'input_type_ids': type_s1,
            'lens':len(data['arg_types']),
            'id_list': id_list},labels)


def gen():
    for _, data_batch in data.groupby(np.arange(len(data))//batch_size):
        if len(data_batch) < batch_size:
            continue
        ids = []

        sentence1 = []
        for _,data_batch_i in data_batch.iterrows():
            print(type(data_batch_i['arg_types']))

            id_list = []
            sentence2 = np.array(transform(data_batch_i['body.1'])[:SEQ_LENGTH-1]+[3])
            for label in eval(data_batch_i['arg_names']):
                print(sentence2, label, np.where(sentence2 == subword_tokenizer.encode(label)[0])[0][0])
                id_list.append( np.where(sentence2 == subword_tokenizer.encode(label)[0])[0][0])
            sentence1.append(sentence2)
        sentence1 = tf.ragged.constant(sentence1)
        sentence1 = sentence1.to_tensor(default_value=0, shape=[batch_size, SEQ_LENGTH])
        id_list =  tf.convert_to_tensor(id_list)
        # sentence1 = tf.reshape(sentence1, [batch_size,8*MAX_BODY_LENGTH])
        labels = tf.convert_to_tensor([np.array(data_batch_i['labels']) for _,data_batch_i in data_batch.iterrows()])
        type_s1 = tf.zeros_like(sentence1)
        yield ({'input_word_ids': sentence1,
            'input_mask': tf.ones_like(sentence1),
            'input_type_ids': type_s1,
            'lens':len(data['arg_types']),
            'id_list': id_list},labels)
next(gen())

<class 'list'>
    @staticmethod
    def to_output(task: dict) -> str:
        """
        Convert serialized task representation to Taskwarrior JSON hook output
        format.

        :param task: serialized task
        :return: Taskwarrior JSON
        """
        fields = Task.FIELDS.copy()

        for k, v in task.items():
            field_type = fields.get(k, None)
            if isinstance(field_type, ArrayField) and not isinstance(field_type, AnnotationArrayField):
                task[k] = ','.join(v)

        return json.dumps(task, separators=(',', ':'))
[    2    34    66   163  1527    15    43   253   432    20  1115    25
   210    19  1716   148    25    15    34    45    86    35    26    63
  3994    17 24350    17  1449    17  4070    17    75    17 28645 21947
 26414   180    17  2058    17  8861    17   530    35    26    63   597
    33    35    26    35    26    63    42   280    17  1449    42    17
 24350    17  1449    35    26    63    42   393    42    1

({'input_word_ids': <tf.Tensor: shape=(1, 512), dtype=int32, numpy=
  array([[    2,    34,    66,   163,  1527,    15,    43,   253,   432,
             20,  1115,    25,   210,    19,  1716,   148,    25,    15,
             34,    45,    86,    35,    26,    63,  3994,    17, 24350,
             17,  1449,    17,  4070,    17,    75,    17, 28645, 21947,
          26414,   180,    17,  2058,    17,  8861,    17,   530,    35,
             26,    63,   597,    33,    35,    26,    35,    26,    63,
             42,   280,    17,  1449,    42,    17, 24350,    17,  1449,
             35,    26,    63,    42,   393,    42,    17, 28645, 21947,
          26414,   180,    17,  2058,    35,    26,    63,    95,    15,
            504,    24,  3858,    21, 15458,    21,   878,    20,    19,
             15,    10,    82,   341,    16,   337,    68,  1115,    21,
            524,    20,    19,    25,    15,    34,    62,  1502,   142,
             24,   504,    21,   157,    20,   341,    1

In [5]:
a =  """
    def main(task: Task) -> None:
        task_tags = task.get('tags', [])

        # Handle case when task has no context tags and no inbox tag.
        if not any('@' in t for t in task_tags) and "in" not in task_tags:
            task_tags.append('in')
            task['tags'] = task_tags
            logger.info("Task had no context tag - inbox tag has be applied")

        # Handle case when task has context tag and inbox tag.
        elif any('@' in t for t in task_tags) and u'in' in task_tags:
            task['tags'].remove('in')
            logger.info("Task had both context and inbox tags - inbox tag has been removed.")"""

# a = '[CLS]'

enc = code_to_subtokenized_sentences.code_to_cubert_sentences(
        code=a,
        initial_tokenizer=tokenizer,
        subword_tokenizer=subword_tokenizer)


In [6]:
subword_tokenizer.encode(tokenizer.tokenize('task')[0])

[1115]

In [7]:
subword_tokenizer.decode_list(subword_tokenizer.encode('task'))

['task_']

In [8]:
subword_tokenizer.encode('task')

[1115]

In [9]:
tokenizer.tokenize('task')

['task', '___EOS___']

In [10]:
enc = next(gen())
print(enc)
bert_encoder(enc[0])

    @staticmethod
    def to_output(task: dict) -> str:
        """
        Convert serialized task representation to Taskwarrior JSON hook output
        format.

        :param task: serialized task
        :return: Taskwarrior JSON
        """
        fields = Task.FIELDS.copy()

        for k, v in task.items():
            field_type = fields.get(k, None)
            if isinstance(field_type, ArrayField) and not isinstance(field_type, AnnotationArrayField):
                task[k] = ','.join(v)

        return json.dumps(task, separators=(',', ':'))
[    2    34    66   163  1527    15    43   253   432    20  1115    25
   210    19  1716   148    25    15    34    45    86    35    26    63
  3994    17 24350    17  1449    17  4070    17    75    17 28645 21947
 26414   180    17  2058    17  8861    17   530    35    26    63   597
    33    35    26    35    26    63    42   280    17  1449    42    17
 24350    17  1449    35    26    63    42   393    42    17 28645 21947
 

[<tf.Tensor: shape=(1, 512, 1024), dtype=float32, numpy=
 array([[[-0.08364937,  0.14709947,  1.245698  , ...,  1.4123988 ,
          -0.31031662,  0.3925744 ],
         [ 1.8032562 ,  0.54278857,  1.4098477 , ..., -0.1873635 ,
           0.7792913 , -1.2085233 ],
         [ 1.607081  , -0.7228891 ,  1.1449388 , ..., -0.14121169,
           0.94634205,  0.01603719],
         ...,
         [ 0.80791605, -1.0993781 , -0.22724296, ..., -0.4450596 ,
           0.03087633,  0.5827752 ],
         [-0.24425574, -0.0615343 ,  0.39945853, ...,  0.00753956,
          -1.8997458 ,  0.20064537],
         [ 0.44453552, -0.20257723,  0.507595  , ..., -1.4222213 ,
          -0.37769067, -0.3870968 ]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
 array([[ 0.84618807, -0.80333686, -0.9597301 , ...,  0.66008353,
         -0.46292236,  0.99918175]], dtype=float32)>]

In [11]:
for i in range(enc[0]['input_word_ids'].shape[0]):
     print(enc[0]['input_word_ids'][i].numpy()[enc[0]['id_list'][i]])
     print(bert_encoder(enc[0])[0][i][enc[0]['id_list'][i]])

1115
tf.Tensor(
[ 0.8117472   0.45972186  1.5196487  ...  0.01904918 -1.066282
  0.363116  ], shape=(1024,), dtype=float32)


In [12]:
subword_tokenizer.decode_list(enc[0]['input_word_ids'].numpy()[0])

['[CLS]_',
 '\\u\\u\\uINDENT\\u\\u\\u ',
 '   _',
 '@_',
 'staticmethod_',
 '\\u\\u\\uNEWLINE\\u\\u\\u_',
 'def_',
 'to\\u^_',
 'output_',
 '(_',
 'task_',
 ':_',
 'dict_',
 ')_',
 '->_',
 'str_',
 ':_',
 '\\u\\u\\uNEWLINE\\u\\u\\u_',
 '\\u\\u\\uINDENT\\u\\u\\u ',
 '       _',
 '"""^_',
 '\\u\\u\\uNLCHAR\\u\\u\\u^',
 '_',
 '        ^_',
 'Convert^_',
 ' ^_',
 'serialized^_',
 ' ^_',
 'task^_',
 ' ^_',
 'representation^_',
 ' ^_',
 'to^_',
 ' ^_',
 'Tas',
 'kwa',
 'rri',
 'or^_',
 ' ^_',
 'JSON^_',
 ' ^_',
 'hook^_',
 ' ^_',
 'output^_',
 '\\u\\u\\uNLCHAR\\u\\u\\u^',
 '_',
 '        ^_',
 'format^_',
 '.^_',
 '\\u\\u\\uNLCHAR\\u\\u\\u^',
 '_',
 '\\u\\u\\uNLCHAR\\u\\u\\u^',
 '_',
 '        ^_',
 ':^_',
 'param^_',
 ' ^_',
 'task^_',
 ':^_',
 ' ^_',
 'serialized^_',
 ' ^_',
 'task^_',
 '\\u\\u\\uNLCHAR\\u\\u\\u^',
 '_',
 '        ^_',
 ':^_',
 'return^_',
 ':^_',
 ' ^_',
 'Tas',
 'kwa',
 'rri',
 'or^_',
 ' ^_',
 'JSON^_',
 '\\u\\u\\uNLCHAR\\u\\u\\u^',
 '_',
 '        ^_',
 '"""_',
 '\\u\\

In [13]:
enc[0]['id_list'].numpy()

array([10], dtype=int32)

In [14]:
subword_tokenizer.encode('i')

[127]

In [12]:
text = 'input_stream'
subword_tokenizer.encode(text)

[708, 222, 1353]

In [15]:
a = tokenizer.tokenize('input_stream')
print(a)
sum([subword_tokenizer.encode_without_tokenizing(i) for i in a],[])


['input_^', 'stream', '___EOS___']


[690, 1353, 121]

In [33]:
for _,i in data.iterrows():
    print(i['arg_types'][0])

rgparse.ArgumentParser
argparse._SubParsersAction
argparse.ArgumentParser
argparse._SubParsersAction
Task
Task
str
List[Task]
pandas.DataFrame
List[Task]
argparse.ArgumentParser
argparse._SubParsersAction
annofabapi.Resource
Callable[..., Any]
str
str
str
argparse.ArgumentParser
argparse._SubParsersAction
argparse.ArgumentParser
argparse._SubParsersAction
pandas.DataFrame
List[Path]
argparse.ArgumentParser
argparse._SubParsersAction
pandas.DataFrame
TaskStatus
Task
List[Task]
argparse.ArgumentParser
argparse._SubParsersAction
argparse.ArgumentParser
argparse._SubParsersAction
argparse.ArgumentParser
argparse._SubParsersAction
annofabapi.AnnofabApi
List[Task]
str
str
Dict[str, Any]
str
List[Task]
Optional[Dict[str, Any]]
Dict[str, Any]
str
pandas.DataFrame
List[TaskHistory]
List[TaskHistory]
List[TaskHistory]
List[TaskHistory]
str
List[Task]
pandas.DataFrame
pandas.DataFrame
pandas.DataFrame
pandas.DataFrame
pandas.DataFrame
pandas.DataFrame
pandas.DataFrame
pandas.DataFrame
pandas.Data

KeyboardInterrupt: 